In [323]:
import pandas as pd

In [324]:
file_path = r"C:\Windows\System32\battery-report.html"
history = pd.read_html(file_path)[2]
history = history[history['STATE'] != 'Suspended'].drop(["CAPACITY REMAINING.1", "STATE"], axis = 1)
map = {
    "AC": 1,
    "Battery": 0
}
history["SOURCE"] = history["SOURCE"].map(map).astype(float)
history["CAPACITY REMAINING"] = history["CAPACITY REMAINING"].str[:-2].astype(float)
dt_full = pd.to_datetime(history['START TIME'], errors = "coerce")
history["DATE"] = dt_full.dt.day_name().ffill()
history['TIME'] = pd.to_datetime(history['START TIME'], format = "mixed")
map2 = {
    "Sunday": 1,
    "Monday": 2,
    "Tuesday": 3,
    "Wednesday": 4,
    "Thursday": 5,
    "Friday": 6,
    "Saturday": 7
}
history = history.drop("START TIME", axis = 1)
history["DATE"] = history["DATE"].map(map2).astype(float)
history["TIME"] = history["TIME"].dt.second + history["TIME"].dt.minute * 60 + history["TIME"].dt.hour * 3600
history

,SOURCE,CAPACITY REMAINING,DATE,TIME
0,1.0,69.0,7.0,33382
1,0.0,99.0,7.0,37033
2,1.0,99.0,7.0,37086
3,1.0,100.0,7.0,40005
4,1.0,100.0,7.0,40302
...,...,...,...,...
204,0.0,55.0,5.0,85741
205,0.0,55.0,5.0,85862
207,0.0,53.0,5.0,31342
208,0.0,53.0,5.0,31377


In [325]:
import torch
class chargenet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 8),
            torch.nn.ReLU(),
            torch.nn.Linear(8, 2)
        )
    def forward(self, x):
        return self.net(x)
model = chargenet()
lossfn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

In [326]:
X = torch.from_numpy(history.drop("SOURCE", axis = 1).astype("float32").values)
Y = torch.from_numpy(history["SOURCE"].astype("int64").values)

In [327]:
# import sklearn
# X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [328]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train)), batch_size=32, shuffle=True)
# test_loader  = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(X_test),  torch.from_numpy(y_test)), batch_size=32)

In [329]:
for i in range(500):
    model.train()
    optimizer.zero_grad()
    pred = model(X)
    loss = lossfn(pred, Y)
    loss.backward()
    optimizer.step()
    print(f"epoch #{i} has loss {loss.item()}")

epoch #0 has loss 63.48090362548828
epoch #1 has loss 31.661209106445312
epoch #2 has loss 1.529861330986023
epoch #3 has loss 33.51656723022461
epoch #4 has loss 2.728313684463501
epoch #5 has loss 8.770719528198242
epoch #6 has loss 9.710408210754395
epoch #7 has loss 6.627087116241455
epoch #8 has loss 1.3850206136703491
epoch #9 has loss 12.937883377075195
epoch #10 has loss 2.8065879344940186
epoch #11 has loss 5.903985023498535
epoch #12 has loss 5.947465419769287
epoch #13 has loss 3.320810317993164
epoch #14 has loss 5.828377723693848
epoch #15 has loss 2.155888319015503
epoch #16 has loss 3.4570281505584717
epoch #17 has loss 2.4300696849823
epoch #18 has loss 2.8862698078155518
epoch #19 has loss 2.2206637859344482
epoch #20 has loss 2.8215491771698
epoch #21 has loss 1.5522602796554565
epoch #22 has loss 4.834412097930908
epoch #23 has loss 3.0784695148468018
epoch #24 has loss 5.045281410217285
epoch #25 has loss 4.777444362640381
epoch #26 has loss 2.4996330738067627
epoch

In [330]:
model.eval()
correct = total = 0
with torch.no_grad():
    logits = model(X)
    preds = logits.argmax(dim=1)
    correct += (preds == Y).sum().item()
    total += Y.size(0)
print(f"Test Accuracy: {correct/total*100:.2f}%")

Test Accuracy: 71.21%
